In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
#Para implementacion, se descarga la libreria y en este caso se agrega .jar a la libreria de spark
pyspark.__version__

'3.3.2'

In [3]:
credentials_location = "D:\Zoomcamp\\2023-de-zoomcamp\data\lib\dtc-de-zoom-7fbf81b52a44.json"
#.set("spark.jars", "../data/lib/gcs-connector-hadoop3-latest.jar") \

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [4]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

#hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df_green = spark.read.option('header',True).csv('gs://etl-de-zoom-dbn/green/green_tripdata_2019-01.csv.gz')

In [14]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime','tpep_pickup_datetime')
df_green = df_green.withColumnRenamed('lpep_dropoff_datetime','tpep_dropoff_datetime')


In [7]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [20]:
comon_columns = []

for col in df_green.columns:
    if col in df_yellow.columns:
        comon_columns.append(col)
        

In [22]:
from pyspark.sql import functions as F

In [29]:
df_green_sel = df_green.select(comon_columns)\
    .withColumn('service_type',F.lit('green'))

In [28]:
df_yellow_sel = df_green.select(comon_columns)\
    .withColumn('service_type',F.lit('yellow'))

In [30]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [31]:
df_trips_data

DataFrame[VendorID: bigint, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: double, PULocationID: bigint, DOLocationID: bigint, passenger_count: double, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, payment_type: double, congestion_surcharge: double, service_type: string]

In [33]:
print(df_green_sel.count(),
      df_yellow_sel.count(),
      df_trips_data.count())

139013 139013 278026


In [35]:
df_trips_data.groupBy('service_type').count().show()

+------------+------+
|service_type| count|
+------------+------+
|       green|139013|
|      yellow|139013|
+------------+------+



In [37]:
df_trips_data.createGlobalTempView('df_trips')

In [39]:
spark.sql('''
    SELECT * FROM df_trips LIMIT 10
''').show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2| 2020-12-31 19:15:56|  2020-12-31 19:19:52|                 N|       1.0|          43|         151|            1.0|         1.01|        5.5|  0.5|    0.5|       0.0|        

In [23]:
spark.stop()